In [1]:
import os
os.chdir("..")

# MLP - Multi-Layer Perceptron

In this notebook, I create a deep learning model to predict the `diameter` and `albedo` values. Then save the model for use in `data-cleaning` notebook`.

## Designing the Architecture

In [2]:
from typing import List

import torch
from torch.nn import LazyLinear, Dropout, ReLU, Sequential, LayerNorm
from torch.nn import HuberLoss

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(29)

In [3]:
class SimpleBlock(torch.nn.Module):
    def __init__(self, num_output: int, dropout: float, device: str = "cuda"):
        super().__init__()

        self.net = Sequential(
            LazyLinear(num_output, device=device),
            LayerNorm(num_output, device=device),
            ReLU(),
            Dropout(dropout),
        )

    def __call__(self, X):
        return self.net(X)


block = SimpleBlock(64, 0.2, device=device)
test = torch.arange(64, dtype=torch.float32).reshape(-1, 8).to(device)
block(test)

/home/ishrak/Documents/Asteroid-Mining-Analysis/.venv/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4527e-01,
         2.4261e-01, 4.9356e-01, 0.0000e+00, 0.0000e+00, 7.1284e-01, 0.0000e+00,
         6.8023e-01, 0.0000e+00, 8.1596e-01, 3.1018e-01, 0.0000e+00, 1.4151e+00,
         1.8673e+00, 9.4119e-01, 0.0000e+00, 1.8731e+00, 1.3865e+00, 0.0000e+00,
         0.0000e+00, 6.8817e-01, 6.1017e-01, 0.0000e+00, 3.4964e-01, 0.0000e+00,
         1.4606e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4808e+00, 0.0000e+00,
         1.9103e-01, 2.7640e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.3514e+00,
         0.0000e+00, 5.9760e-01, 4.5625e-01, 0.0000e+00, 1.9209e-01, 6.2832e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 2.5578e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.1210e-01,
         1.1073e+00, 1.2956e+00, 1.9248e+00, 8.8449e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4441e-01,
         7.1475e-01, 0.0000e+00, 0.0000e+00, 0.0000

In [4]:
class Block(torch.nn.Module):
    def __init__(
        self,
        num_output_list: List[int],
        dropout_list: List[float],
        num_output: int,
        device: str = "cuda",
    ) -> None:
        super().__init__()

        self.layers = [
            SimpleBlock(no, d, device) for no, d in zip(num_output_list, dropout_list)
        ]

        self.layer_out = LazyLinear(num_output, device=device)

    def __call__(self, X):
        output = X

        for layer in self.layers:
            output = torch.concat([output, layer(X)], axis=1)

        return self.layer_out(output)


block = Block(num_output_list=[32, 16, 16], dropout_list=[0.3, 0.2, 0.2], num_output=64, device=device)
block(test).shape

torch.Size([8, 64])

In [5]:
class MLP(torch.nn.Module):
    def __init__(
        self,
        n: int,
        num_output_list: List[int],
        dropout_list: List[int],
        block_io_shape: int = 64,
        device: str = "cuda",
    ):
        super().__init__()

        self.layer_in = LazyLinear(block_io_shape, device=device)

        self.layers = [
            Block(
                num_output_list=num_output_list,
                dropout_list=dropout_list,
                num_output=block_io_shape,
            )
            for _ in range(n)
        ]

        self.output_layer = LazyLinear(2, device=device)

        self.apply(self._init)

        self.mse_loss = HuberLoss()

    def _init(self, module):
        if type(module) is torch.nn.Linear:
            torch.nn.init.xavier_uniform_(module.weight)
            torch.nn.init.zeros_(module.bias)

    def __call__(self, X):
        output = self.layer_in(X)

        for l in self.layers:
            output = l(output) + output

        return self.output_layer(output)

    def loss(self, y, pred):
        return self.mse_loss(y, pred)


mlp = MLP(3, num_output_list=[32, 16, 16], dropout_list=[0.3, 0.2, 0.2], device=device)
mlp(test)

tensor([[ -2.5064,  -1.2346],
        [ -5.5732,  -3.1250],
        [ -8.5703,  -4.5417],
        [-11.8984,  -5.7778],
        [-15.1231,  -7.5824],
        [-17.9572,  -8.6474],
        [-21.2816, -10.4809],
        [-24.2204, -11.7324]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [6]:
del mlp, test, block

## Setup Dataset

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("data/Asteroid_Imputed.csv")
df.shape

/tmp/ipykernel_7098/4156258339.py:1: DtypeWarning: Columns (9,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/Asteroid_Imputed.csv")


(839714, 21)

In [9]:
df.columns

Index(['a', 'e', 'i', 'om', 'w', 'q', 'ad', 'per_y', 'data_arc',
       'condition_code', 'n_obs_used', 'H', 'neo', 'pha', 'diameter', 'albedo',
       'moid', 'class', 'n', 'per', 'ma'],
      dtype='object')

In [10]:
df.drop(df[(df.condition_code == 'D') | (df.condition_code == 'E')].index, inplace=True)

In [11]:
df.condition_code = df.condition_code.map(lambda x: int(x))

In [12]:
df_processed = df.copy(deep=True)

Normalizing numerical columns using min-max scaling.

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
categorical_columns = ["pha", "neo", "condition_code", "class"]
numerical_columns = df.columns[~df.columns.isin(categorical_columns)]
print(numerical_columns)

Index(['a', 'e', 'i', 'om', 'w', 'q', 'ad', 'per_y', 'data_arc', 'n_obs_used',
       'H', 'diameter', 'albedo', 'moid', 'n', 'per', 'ma'],
      dtype='object')


Before **Min-Max Normalization**

I won't be normalizing the values for `diameter` and `albedo`, because I want the deep learning model to predict these values.

In [15]:
numerical_columns = numerical_columns.drop(["diameter", "albedo"])

In [16]:
df_processed[numerical_columns].sample(3)

,a,e,i,om,w,q,ad,per_y,data_arc,n_obs_used,H,moid,n,per,ma
474041,2.623381,0.030128,15.429898,90.119096,309.321053,5.164125,2.696228,4.280620,3209.00,96,16.700,4.195230,0.231959,1551.995156,151.953110
757496,2.612485,0.167968,4.622640,20.160320,284.182470,2.107553,3.108307,5.487518,3557.54,25,16.811,1.102941,0.233412,1542.336822,162.250462
381962,3.082989,0.064013,10.494348,5.944069,328.582692,2.901878,3.507587,3.742927,4123.00,107,16.400,1.898150,0.182073,1977.226312,350.458740


In [17]:
for column in numerical_columns:
    values = df_processed[column].values.reshape(-1, 1)
    df_processed[column] = MinMaxScaler().fit_transform(values)

After **Min-Max Normalization**

In [18]:
df_processed[numerical_columns].sample(3)

,a,e,i,om,w,q,ad,per_y,data_arc,n_obs_used,H,moid,n,per,ma
737743,0.971669,0.118442,0.017659,0.243951,0.368326,0.031756,0.000398,0.000021,0.067829,0.009224,0.555335,0.020320,0.098652,0.000023,0.186316
355844,0.971671,0.142601,0.038003,0.187247,0.806842,0.022292,0.000436,0.000025,0.065256,0.015553,0.518950,0.010945,0.088616,0.000025,0.293588
837033,0.971658,0.291284,0.062758,0.098827,0.039972,0.005308,0.000188,0.001585,0.000096,0.009654,0.793003,0.000092,0.233819,0.000008,0.145772


One hot encode categorical columns.

In [19]:
df_processed = pd.get_dummies(
    df_processed,
    columns=categorical_columns,
    dummy_na=True,
)
df_processed.shape

(839559, 49)

In [20]:
df_processed.columns

Index(['a', 'e', 'i', 'om', 'w', 'q', 'ad', 'per_y', 'data_arc', 'n_obs_used',
       'H', 'diameter', 'albedo', 'moid', 'n', 'per', 'ma', 'pha_N', 'pha_Y',
       'pha_nan', 'neo_N', 'neo_Y', 'neo_nan', 'condition_code_0.0',
       'condition_code_1.0', 'condition_code_2.0', 'condition_code_3.0',
       'condition_code_4.0', 'condition_code_5.0', 'condition_code_6.0',
       'condition_code_7.0', 'condition_code_8.0', 'condition_code_9.0',
       'condition_code_nan', 'class_AMO', 'class_APO', 'class_AST',
       'class_ATE', 'class_CEN', 'class_HYA', 'class_IEO', 'class_IMB',
       'class_MBA', 'class_MCA', 'class_OMB', 'class_PAA', 'class_TJN',
       'class_TNO', 'class_nan'],
      dtype='object')

Now, I'll split them into two sets. 

* **Set 1**: Diameter and albedo are both not null. These will be used for training purposes.

* **Set 2**: Diameter or albedo are null. These will be used for inferencing purposes. The model will predict their values given the other column values, and then I'll use the predicted values to impute the missing values.

In [21]:
set_1 = df_processed[df_processed.diameter.notnull() & df_processed.albedo.notnull()]
set_2 = df_processed[~df_processed.isin(set_1)]

print(f"Number of examples for training purposes: {set_1.shape[0]}")
print(f"Number of examples for inference purposes: {set_2.shape[0]}")

Number of examples for training purposes: 136406
Number of examples for inference purposes: 839559


I'll now split the first set into

1. **Training set**: Used specifically to train the model.
2. **Validation set**: Used to check model's performance on unseen data.
3. **Test set**: Used to check generalization performance.

In [22]:
import numpy as np

In [23]:
train = set_1.sample(frac=0.9, random_state=29)
valid = set_1[~set_1.isin(train)].dropna()
test = valid.sample(frac=0.5)
valid = valid[~valid.isin(test)].dropna()

train.shape, valid.shape, test.shape

((122765, 49), (6821, 49), (6820, 49))

In [24]:
train_X = train.drop(columns=["diameter", "albedo"]).values.astype(np.float64)
test_X = valid.drop(columns=["diameter", "albedo"]).values.astype(np.float64)
valid_X = test.drop(columns=["diameter", "albedo"]).values.astype(np.float64)

print(f"Training set X shape: {train_X.shape}")
print(f"Validation set X shape: {valid_X.shape}")
print(f"Test set X shape: {test_X.shape}")

Training set X shape: (122765, 47)
Validation set X shape: (6820, 47)
Test set X shape: (6821, 47)


In [25]:
train_Y = train[["diameter", "albedo"]].values.astype(np.float64)
valid_Y = valid[["diameter", "albedo"]].values.astype(np.float64)
test_Y = test[["diameter", "albedo"]].values.astype(np.float64)

print(f"Training set Y shape: {train_Y.shape}")
print(f"Validation set Y shape: {valid_Y.shape}")
print(f"Test set Y shape: {test_Y.shape}")

Training set Y shape: (122765, 2)
Validation set Y shape: (6821, 2)
Test set Y shape: (6820, 2)


In [26]:
del train, valid, test

Creating a Data Loader for batch processing

In [27]:
from torch.utils.data import DataLoader

In [28]:
TRAIN_BATCH_SIZE = 256
VALIDATION_BATCH_SIZE = 256
TEST_BATCH_SIZE = 256

In [29]:
train_X = torch.tensor(train_X, dtype=torch.float32)
train_Y = torch.tensor(train_Y, dtype=torch.float32)

valid_X = torch.tensor(valid_X, dtype=torch.float32)
valid_Y = torch.tensor(valid_Y, dtype=torch.float32)

test_X = torch.tensor(test_X, dtype=torch.float32)
test_Y = torch.tensor(test_Y, dtype=torch.float32)

In [30]:

train_loader = DataLoader(
    list(zip(train_X, train_Y)),
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
)

valid_loader = DataLoader(
    list(zip(valid_X, valid_Y)),
    shuffle=True,
    batch_size=VALIDATION_BATCH_SIZE,
)

test_loader = DataLoader(
    list(zip(test_X, test_Y)),
    shuffle=True,
    batch_size=TEST_BATCH_SIZE,
)

## Training Script

In [31]:
import pickle

from torch.optim import AdamW, lr_scheduler
from tqdm import tqdm

In [70]:
model = MLP(3, num_output_list=[256, 128, 64], dropout_list=[0.3, 0.2, 0.1], device=device)
optimizer = AdamW(model.parameters(), lr=4e-6)
scheduler = lr_scheduler.ExponentialLR(
    optimizer,
    gamma=0.9,
    last_epoch=-1,
    verbose=False,
)

num_epochs = 100
patience = 10
min_eval_loss = float("inf")
early_stopping_hook = 0
tracking_information = []
scaler = torch.cuda.amp.GradScaler()
root_save_dir = "model_dir"

In [71]:
def epoch_train():
    train_loss = 0
    model.train()

    for _, (X, y) in zip(
        tqdm(range(len(train_loader)), desc="Training batch: ..."),
        train_loader,
    ):
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)

        loss = model.loss(y, pred)
        train_loss += loss.item()
        optimizer.zero_grad()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
    return train_loss

In [72]:
def epoch_validate():
    model.eval()
    valid_loss = 0

    for _, batch in zip(
        tqdm(range(len(valid_loader)), desc="Validating batch: ..."),
        valid_loader,
    ):
        X, y = batch
        X = X.to(device)
        y = y.to(device)

        pred = model(X)

        loss = model.loss(y, pred)
        valid_loss += loss.item()
        
    return valid_loss

In [73]:
def handle_tracking(epoch, train_loss, valid_loss):
    tracking_information.append(
        (
            train_loss / len(train_loader),
            valid_loss / len(valid_loader),
            optimizer.param_groups[0]["lr"],
        )
    )
    print(
        "Epoch: {} - Training loss: {} - Eval Loss: {} - LR: {}".format(
            epoch + 1,
            train_loss / len(train_loader),
            valid_loss / len(valid_loader),
            optimizer.param_groups[0]["lr"],
        )
    )

In [74]:
for epoch in range(num_epochs):
    train_loss = epoch_train()
    valid_loss = epoch_validate()

    handle_tracking(epoch, train_loss, valid_loss)
    scheduler.step()

    if valid_loss < min_eval_loss:
        model_save_file = os.path.join(root_save_dir, "resnet_model")
        torch.save(model.state_dict(), model_save_file)
        print(f"Saved model to {model_save_file}")
        min_eval_loss = valid_loss
        early_stopping_hook = 0
    else:
        early_stopping_hook += 1

        if early_stopping_hook > patience:
            break


pickle.dump(
    tracking_information,
    open(os.path.join(root_save_dir, "tracking_information.pkl"), "wb"),
)
print("The training process has done!")

Training batch: ...:   0%|          | 0/480 [00:00<?, ?it/s]

Validating batch: ...: 100%|██████████| 27/27 [00:00<00:00, 759.81it/s]


Epoch: 1 - Training loss: 1.9524196463326613 - Eval Loss: 1.8300088290815 - LR: 4e-06
Saved model to model_dir/resnet_model


Validating batch: ...: 100%|██████████| 27/27 [00:00<00:00, 753.82it/s]


Epoch: 2 - Training loss: 1.855955681949854 - Eval Loss: 1.7394157250722249 - LR: 3.6e-06
Saved model to model_dir/resnet_model


...

Epoch: 55 - Training loss: 1.290819305429856 - Eval Loss: 1.2276332334235862 - LR: 1.3525567654090923e-08


Validating batch: ...: 100%|██████████| 27/27 [00:00<00:00, 746.39it/s]


Epoch: 56 - Training loss: 1.2901231071601311 - Eval Loss: 1.2214339463799089 - LR: 1.2173010888681831e-08


Validating batch: ...: 100%|██████████| 27/27 [00:00<00:00, 733.23it/s]

Epoch: 57 - Training loss: 1.2897595042983692 - Eval Loss: 1.2262143912138763 - LR: 1.0955709799813649e-08
The training process has done!
